## Imports

In [1]:
from src.services.stt.stt import stt_from_audio
from src.services.assistant.assistant import Assistant
from src.services.speaker_recognition.speaker_recognition import verify_speakers

/Users/thiago/Documents/GitHub/EchoLoco/.venv311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## STT

In [4]:
texto = stt_from_audio('audios/fibonnaci.wav')

Device set to use cpu
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [5]:
texto

'Olá, eu queria muito uma função recursiva que faz Fibonat em Python.'

## Speaker Verification

In [2]:
from src.services.speaker_recognition.speaker_recognition import load_model, verify_speakers

file1 = "audios/audio_teste.wav"
file2 = "audios/fibonnaci.wav"

model_speaker_recognition = load_model()
result = verify_speakers(model_speaker_recognition, file1, file2)
print(result)

[NeMo W 2025-06-19 11:13:00 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2025-06-19 11:13:00 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data

[NeMo I 2025-06-19 11:13:00 nemo_logging:393] PADDING: 16
[NeMo I 2025-06-19 11:13:01 nemo_logging:393] Model EncDecSpeakerLabelModel was successfully restored from /Users/thiago/.cache/huggingface/hub/models--nvidia--speakerverification_en_titanet_large/snapshots/0dc382f40121a5fbd34db10a2bb04d826c2be6a8/speakerverification_en_titanet_large.nemo.
Modelo carregado: EncDecSpeakerLabelModel(
  (loss): AngularSoftmaxLoss()
  (eval_loss): AngularSoftmaxLoss()
  (_accuracy): TopKClassificationAccuracy()
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ConvASREncoder(
    (encoder): Sequential(
      (0): JasperBlock(
        (mconv): ModuleList(
          (0): MaskedConv1d(
            (conv): Conv1d(80, 80, kernel_size=(3,), stride=(1,), padding=(1,), groups=80, bias=False)
          )
          (1): MaskedConv1d(
            (conv): Conv1d(80, 1024, kernel_size=(1,), stride=(1,), bias=False)
          )
          (2): BatchNorm1d(

## Assistant

In [4]:
assistant = Assistant()
resposta = assistant.reply(texto)
print("EchoLoco:", resposta)

EchoLoco: Olá! A sequência de Fibonacci é uma série numérica onde cada número é a soma dos dois anteriores, começando geralmente com 0 e 1. A função recursiva para calcular o n-ésimo termo dessa sequência em Python é bem simples.

Aqui está um exemplo de como você pode implementá-la:

```python
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

# Exemplo de uso
n = 10  # Para encontrar o 10º termo
print(f"O {n}º termo da sequência de Fibonacci é: {fibonacci(n)}")
```

### Explicação:
1. **Base da Recursão**: A função verifica se n é 0 ou 1, retornando 0 ou 1 respectivamente. Essas são as condições de parada.
2. **Chamada Recursiva**: Se n for maior que 1, a função chamará a si mesma duas vezes: uma para (n-1) e outra para (n-2), somando os resultados.

### Considerações:
- Essa implementação é simples, mas não é a mais eficiente, pois recalcula valores repetidamente. Para casos maiores de n, 

## TTS

In [6]:
from src.services.tts.tts import load_tts_pipeline, tts_from_text

tts_pipe, language, output_dir, audio_format = load_tts_pipeline()
audio_path = tts_from_text(
    "vai tomar no cu Daniel filho da puta",
    tts_pipe,
    language=language,
    output_dir=output_dir,
    file_name="tts_output",
    audio_format=audio_format
)
print(f"Áudio gerado em: {audio_path}")

Áudio gerado em: ./audios_sintetizados/tts_output.wav


In [8]:
from IPython.display import Audio

Audio(audio_path)

## Qdrant (banco de dados vetorial)

In [ ]:
'''Criação do banco de dados vetorial Qdrant (tem que dar um docker compose up antes de rodar isso)'''

from src.services.vector_database.qdrant_service import QdrantService

svc = QdrantService()
svc.create_collection(force_recreate=True)

In [ ]:
'''Teste de inserção e busca de embeddings'''

import numpy as np

emb1 = np.random.rand(svc.vector_size).tolist()
emb2 = np.random.rand(svc.vector_size).tolist()

svc.insert_embedding(emb1, payload={"name": "Alice", "age": 30})
svc.insert_embedding(emb2, payload={"name": "Thiago", "age": 30})

res = svc.search_similar(emb2, top_k=2)
print("Resultados:", res)

Resultados: [ScoredPoint(id='f8f27f0a-4861-455a-97f1-f6147f995a7a', version=1, score=1.0000001, payload={'age': 30, 'name': 'Thiago'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='de999151-066b-45f0-8495-e53cb4e1af3f', version=0, score=0.7418833, payload={'age': 30, 'name': 'Alice'}, vector=None, shard_key=None, order_value=None)]
